In [ ]:
!pip install transformers torch langchain sentencepiece


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms.base import LLM
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# ------------------------------
# Step 1: Load Hugging Face Model
# ------------------------------
model_name = "gpt2"  # small, CPU-friendly model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Use Hugging Face pipeline for text generation
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=200,
    do_sample=True,
    temperature=0.7
)

# ------------------------------
# Step 2: Wrap as LangChain LLM
# ------------------------------
class HuggingFaceLLM(LLM):
    """LangChain wrapper for Hugging Face transformers"""

    @property
    def _llm_type(self):
        return "huggingface"

    def _call(self, prompt: str, stop=None):
        output = generator(prompt, max_length=200, num_return_sequences=1)
        return output[0]["generated_text"]

# Instantiate the LLM
llm = HuggingFaceLLM()

# ------------------------------
# Step 3: Setup ConversationChain
# ------------------------------
memory = ConversationBufferMemory()
chatbot = ConversationChain(llm=llm, memory=memory)

# ------------------------------
# Step 4: Run Chat Loop
# ------------------------------
print("Chatbot is ready! Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye!")
        break
    response = chatbot.run(user_input)
    print("Chatbot:", response)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu
/tmp/ipython-input-2435167748.py:43: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipython-input-2435167748.py:44: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chatbot = ConversationChain(llm=llm, memory=memory)


Chatbot is ready! Type 'exit' to quit.
You: what is heat


/tmp/ipython-input-2435167748.py:55: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chatbot.run(user_input)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: what is heat
AI: how cold is it

Humans: what is the last time you had a cold

Humans: have you ever?

Humans: why did you go to the other side of the world?

Humans: the other side of the world?

Humans: did it come back?

Human: I never went to the other side.

AI: where did you go to?

Human: I'm talking about the past.

AI: I heard that you all said you were going to do something.

Human: yeah, I am.

AI: and did you tell anyone?

You: exit
Chatbot: Goodbye!
